<a href="https://colab.research.google.com/github/Sahil-1509/Machine_Learning_Projects/blob/main/Rumour_Verification_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re
import nltk
from tensorflow import keras
from gensim.models import Word2Vec
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import PunktSentenceTokenizer
from sklearn.metrics import confusion_matrix, accuracy_score
import tensorflow as tf

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
read_file = pd.read_table('source_tweets.txt')
read_file.to_csv (r'source_tweets.csv', index=None)
dataset_1 = pd.read_csv('source_tweets.csv')
print(dataset_1)

      731166399389962242  \
0     714598641827246081   
1     691809004356501505   
2     693204708933160960   
3     551099691702956032   
4     767223774072676354   
...                  ...   
1484  692004901455556608   
1485  760109079133990912   
1486  500281131057811456   
1487  523098334421319680   
1488  523545090099523584   

     🔥ca kkk grand wizard 🔥 endorses @hillaryclinton #neverhillary #trump2016 URL  
0     an open letter to trump voters from his top st...                            
1     america is a nation of second chances —@potus ...                            
2     brandon marshall visits and offers advice, sup...                            
3     rip elly may clampett: so sad to learn #beverl...                            
4     former 3 doors down guitarist matt roberts has...                            
...                                                 ...                            
1484  .@potus just announced new reforms to address ...                    

In [ ]:
read_file = pd.read_table('label.txt')
read_file.to_csv (r'label.csv', index=None)
dataset_2 = pd.read_csv('label.csv',sep=':')
print(dataset_2)

      unverified  731166399389962242
0     unverified  714598641827246081
1      non-rumor  691809004356501505
2      non-rumor  693204708933160960
3           true  551099691702956032
4      non-rumor  767223774072676354
...          ...                 ...
1484   non-rumor  692004901455556608
1485  unverified  760109079133990912
1486  unverified  500281131057811456
1487       false  523098334421319680
1488  unverified  523545090099523584

[1489 rows x 2 columns]


In [ ]:
headerList = ['answer','sourceid']
dataset_2.to_csv("label.csv", header=headerList, index=False)
dataset_2 = pd.read_csv("label.csv")
print(dataset_2.head())
print()
headerList = ['sourceid','review']
dataset_1.to_csv("source_tweets.csv", header=headerList, index=False)
dataset_1 = pd.read_csv("source_tweets.csv")
print(dataset_1.head())

       answer            sourceid
0  unverified  714598641827246081
1   non-rumor  691809004356501505
2   non-rumor  693204708933160960
3        true  551099691702956032
4   non-rumor  767223774072676354

             sourceid                                             review
0  714598641827246081  an open letter to trump voters from his top st...
1  691809004356501505  america is a nation of second chances —@potus ...
2  693204708933160960  brandon marshall visits and offers advice, sup...
3  551099691702956032  rip elly may clampett: so sad to learn #beverl...
4  767223774072676354  former 3 doors down guitarist matt roberts has...


In [ ]:
# dataset = pd.concat(map(pd.read_csv, ['source_tweets.csv', 'label.csv']),axis=1)
dataset=pd.merge(dataset_1,dataset_2,on='sourceid')

#counting total number of words
print(sum(dataset['review'].str.count(' ')))
#counting no of unique words
print(dataset['review'].nunique())

21566
1427


In [ ]:
print((dataset['review'].str.count(' ')))
print(max(dataset['review'].str.count(' ')))
print(sum(dataset['review'].str.count(' '))/len(dataset))

0       12
1       15
2       15
3       15
4       17
        ..
1484    13
1485     4
1486    17
1487     4
1488    11
Name: review, Length: 1489, dtype: int64
28
14.48354600402955


In [ ]:
dataset.head()

,sourceid,review,answer
0,714598641827246081,an open letter to trump voters from his top st...,unverified
1,691809004356501505,america is a nation of second chances —@potus ...,non-rumor
2,693204708933160960,"brandon marshall visits and offers advice, sup...",non-rumor
3,551099691702956032,rip elly may clampett: so sad to learn #beverl...,true
4,767223774072676354,former 3 doors down guitarist matt roberts has...,non-rumor


In [ ]:
corpus=[]
for i in range(len(dataset)):
  review = re.sub('[^a-zA-Z]', ' ', dataset['review'][i])
  review = review.lower()
  review = review.split()
  lemmatizer = WordNetLemmatizer()
  all_stopwords = stopwords.words('english')
  review = [lemmatizer.lemmatize(word) for word in review if not word in set(all_stopwords)]
  corpus.append(review)  
  review=' '.join(review)
  dataset['review'][i]=review

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [ ]:
dataset.head()
print(dataset.columns)

Index(['sourceid', 'review', 'answer'], dtype='object')


In [ ]:
print(dataset['review'])
print(sum(dataset['review'].str.count(' ')))
print(dataset['review'].nunique())

0       open letter trump voter top strategist turned ...
1       america nation second chance potus new reform ...
2       brandon marshall visit offer advice support br...
3       rip elly may clampett sad learn beverlyhillbil...
4       former door guitarist matt robert died age acc...
                              ...                        
1484    potus announced new reform address overuse sol...
1485                                school satan club url
1486    breaking news according document released pres...
1487                         ebola vaccine url news today
1488    concerned airport passenger suit homemade hazm...
Name: review, Length: 1489, dtype: object
14584
1416


In [ ]:
print(corpus)

[['open', 'letter', 'trump', 'voter', 'top', 'strategist', 'turned', 'defector', 'url', 'via', 'xojanedotcom'], ['america', 'nation', 'second', 'chance', 'potus', 'new', 'reform', 'solitary', 'confinement', 'url', 'url'], ['brandon', 'marshall', 'visit', 'offer', 'advice', 'support', 'brother', 'fallen', 'hero', 'zaevion', 'dobson', 'url', 'url'], ['rip', 'elly', 'may', 'clampett', 'sad', 'learn', 'beverlyhillbillies', 'star', 'donna', 'douglas', 'passed', 'away', 'url'], ['former', 'door', 'guitarist', 'matt', 'robert', 'died', 'age', 'according', 'father', 'url', 'url'], ['craigslist', 'ad', 'get', 'paid', 'hour', 'protest', 'trump', 'rally', 'url', 'url'], ['missing', 'afghan', 'soldier', 'found', 'trying', 'enter', 'canada', 'near', 'niagara', 'fall', 'url', 'url'], ['day', 'ferguson', 'cop', 'told', 'dirty', 'bloody', 'lie', 'via', 'thedailybeast', 'url', 'url'], ['riphulkhogan', 'heart', 'ripping', 'like', 'shirt', 'wwe', 'miss'], ['chick', 'fil', 'manager', 'allegedly', 'banned'

In [ ]:
model = Word2Vec(min_count=5,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     )


In [ ]:
model.build_vocab(corpus, progress_per=1000)
model.train(corpus, total_examples=model.corpus_count, epochs=model.epochs, report_delay=1)

(11732, 80365)

In [ ]:
vocab = list(model.wv.vocab)
print(vocab)
print(len(vocab))

['open', 'letter', 'trump', 'voter', 'top', 'strategist', 'turned', 'defector', 'url', 'via', 'america', 'potus', 'new', 'offer', 'support', 'hero', 'rip', 'elly', 'may', 'clampett', 'sad', 'learn', 'star', 'donna', 'douglas', 'away', 'former', 'died', 'age', 'according', 'ad', 'get', 'paid', 'hour', 'protest', 'missing', 'afghan', 'soldier', 'found', 'trying', 'canada', 'near', 'fall', 'day', 'ferguson', 'cop', 'told', 'heart', 'like', 'shirt', 'chick', 'fil', 'manager', 'banned', 'list', 'slang', 'word', 'islamic', 'tribunal', 'using', 'sharia', 'law', 'texas', 'confirmed', 'rt', 'photo', 'vladimir', 'putin', 'motorcade', 'posted', 'without', 'mh', 'shooter', 'still', 'shot', 'ottawa', 'war', 'memorial', 'parliament', 'taken', 'report', 'american', 'taking', 'year', 'school', 'woman', 'head', 'last', 'night', 'released', 'plan', 'first', 'store', 'keep', 'say', 'bile', 'gold', 'men', 'team', 'rio', 'let', 'friend', 'know', 'state', 'south', 'cut', 'meat', 'beer', 'believe', 'killed',

In [ ]:
vector=model.wv.vectors
vector

array([[-0.05072695,  0.08104977, -0.05636467, ...,  0.0184574 ,
        -0.02163253, -0.0164174 ],
       [-0.01495166,  0.02726618, -0.01668645, ...,  0.00493848,
        -0.00655742, -0.00555997],
       [-0.01607266,  0.02734165, -0.02060434, ...,  0.00738106,
        -0.00733394, -0.00486929],
       ...,
       [-0.00675112,  0.01163138, -0.00938997, ...,  0.00415649,
        -0.00276921, -0.00190965],
       [-0.00129265,  0.00179294, -0.00050877, ...,  0.00095633,
        -0.00209406,  0.00028799],
       [-0.0033877 ,  0.00544167, -0.00406166, ...,  0.00094302,
        -0.00269175, -0.00179486]], dtype=float32)

In [ ]:
model.wv.most_similar(positive=["letter"])

[('url', 0.9890867471694946),
 ('died', 0.9882146120071411),
 ('mojang', 0.9878290891647339),
 ('say', 0.9874548316001892),
 ('people', 0.9874523282051086),
 ('two', 0.9871696829795837),
 ('p', 0.9871575832366943),
 ('least', 0.9870786070823669),
 ('soldier', 0.987033486366272),
 ('ferguson', 0.9870274066925049)]

In [ ]:
x=dataset.iloc[:,1:2].values
y = dataset.iloc[:, -1].values

x.reshape(-1,1)
# print(x)
print(x.shape)
print(x)
y=np.reshape(y,(1489,1))
print(y.shape)


(1489, 1)
[['open letter trump voter top strategist turned defector url via xojanedotcom']
 ['america nation second chance potus new reform solitary confinement url url']
 ['brandon marshall visit offer advice support brother fallen hero zaevion dobson url url']
 ...
 ['breaking news according document released press mikebrown connected earlier robbery ferguson url']
 ['ebola vaccine url news today']
 ['concerned airport passenger suit homemade hazmat flight url url']]
(1489, 1)


In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
print(y)
from collections import Counter
items = Counter(y).keys()
len(items)

[3 1 1 ... 3 0 3]


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


4

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
tokenizer = Tokenizer()
tokenizer.fit_on_texts(dataset['review'])
train_sequences = tokenizer.texts_to_sequences(dataset['review']) #Converting text to a vector of word indexes
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))
print('1st token-id sequnce', train_sequences[0])

Found 4180 unique tokens.
1st token-id sequnce [260, 53, 28, 423, 261, 262, 424, 589, 1, 32, 1264]


In [ ]:
train_sequences = tokenizer.texts_to_sequences(dataset['review']) 
print(dataset['review'][0])
print(text_to_word_sequence(dataset['review'][0]))
print()
tokenizer.num_words = None # take all the tokens
print(tokenizer.texts_to_sequences(dataset['review'])[0])
tokenizer.num_words = 100
print(tokenizer.texts_to_sequences(dataset['review'])[0])

open letter trump voter top strategist turned defector url via xojanedotcom
['open', 'letter', 'trump', 'voter', 'top', 'strategist', 'turned', 'defector', 'url', 'via', 'xojanedotcom']

[260, 53, 28, 423, 261, 262, 424, 589, 1, 32, 1264]
[53, 28, 1, 32]


In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
print(sum(dataset['review'].str.count(' '))/len(dataset))

9.794492948287441


In [ ]:
MAX_SEQUENCE_LENGTH = 10 
trainvalid_data_pre = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH, 
                                padding='pre',
                                truncating='pre',)
print(trainvalid_data_pre.shape)
# type(trainvalid_data_pre)

(1489, 10)


In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='constant')
imputer.fit(trainvalid_data_pre)
trainvalid_data_pre = imputer.transform(trainvalid_data_pre)

print(trainvalid_data_pre.shape)

(1489, 10)


In [ ]:
train_embedding_weights= np.zeros((len(word_index)+1, 300))
for word,index in word_index.items():
  train_embedding_weights[index,:] = model.wv[word] if word in model.wv else np.random.rand(300)
print(train_embedding_weights.shape)

(4181, 300)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(trainvalid_data_pre, y, test_size = 0.2, random_state = 1)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(1191, 10)
(298, 10)
(1191,)
(298,)


In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[32 19  8 18]
 [19 38  6 16]
 [16  5 33 26]
 [21 16  7 18]]


0.40604026845637586

In [ ]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf',degree=5, random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[16 19 30 12]
 [13 46  5 15]
 [13 16 37 14]
 [14 15 19 14]]


0.37919463087248323

In [ ]:
from sklearn.tree import DecisionTreeClassifier
classifier = DecisionTreeClassifier(criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[23 21 17 16]
 [10 36 11 22]
 [13  9 35 23]
 [18  9  8 27]]


0.40604026845637586

In [ ]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[ 6 22 44  5]
 [ 8 47 20  4]
 [15 17 46  2]
 [11 17 32  2]]


0.3389261744966443

In [ ]:
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')
from spacy.matcher import Matcher 
from spacy.tokens import Span 
from tqdm import tqdm

In [ ]:
from sklearn.model_selection import train_test_split
review = dataset['review'].values
label = dataset['answer'].values
label = le.fit_transform(label)
review_train, review_test, label_train, label_test = train_test_split(
review, label, test_size=0.25, random_state=1000)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
review_vectorizer = CountVectorizer()
review_vectorizer.fit(review_train)
Xlr_train = review_vectorizer.transform(review_train)
Xlr_test  = review_vectorizer.transform(review_test)
Xlr_train 

<1116x3546 sparse matrix of type '<class 'numpy.int64'>'
	with 11313 stored elements in Compressed Sparse Row format>

In [ ]:
print(Xlr_train.shape)
print(Xlr_test.shape)
print(label_train.shape)
print(label_test.shape)

(1116, 3546)
(373, 3546)
(1116,)
(373,)


In [ ]:
from collections import Counter
items = Counter(label_train).keys()
print(len(items))
items1 = Counter(label_test).keys()
print(len(items1))

4
4


In [ ]:
from sklearn.linear_model import LogisticRegression
LRmodel = LogisticRegression()
LRmodel.fit(Xlr_train, label_train)
score = LRmodel.score(Xlr_test, label_test)
print("Accuracy:", score) 

Accuracy: 0.8310991957104558


In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(Xlr_train, label_train)
score1 = classifier.score(Xlr_test, label_test)
print("Accuracy:", score1) 
# print("Accuracy:", score) 
# label_pred = classifier.predict(Xlr_test)
# cm = confusion_matrix(label_test, label_pred)
# print(cm)
# accuracy_score(y_test, y_pred)

Accuracy: 0.774798927613941


In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=4200)
tokenizer.fit_on_texts(review_train)
Xcnn_train = tokenizer.texts_to_sequences(review_train)
Xcnn_test = tokenizer.texts_to_sequences(review_test)
vocab_size = len(tokenizer.word_index) + 1  
print(review_train[1])
print(Xcnn_train[1]) 

realdonaldtrump son eric slam hillaryclinton potus response flooding louisiana url
[442, 286, 542, 716, 287, 32, 717, 980, 543, 1]


In [ ]:
print(np.shape(Xcnn_train))
print(np.shape(label_train))
print(np.shape(Xcnn_test))
print(np.shape(label_test))

(1116,)
(1116,)
(373,)
(373,)


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:1970: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


In [ ]:
from keras.preprocessing.sequence import pad_sequences
maxlen = 50
Xcnn_train = pad_sequences(Xcnn_train, padding='post', maxlen=maxlen)
Xcnn_test = pad_sequences(Xcnn_test, padding='post', maxlen=maxlen)
print(Xcnn_train[0, :])
len(Xcnn_train)

[ 128   25  540  541   46  205  174 1575  715    1    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0]


1116

In [ ]:
print(np.shape(Xcnn_train))
print(np.shape(label_train))
print(np.shape(Xcnn_test))
print(np.shape(label_test))

(1116, 50)
(1116,)
(373, 50)
(373,)


In [ ]:
from tensorflow.keras.utils import to_categorical
# Xcnn_train = to_categorical(label_train)
Xcnn_test= to_categorical(Xcnn_test)
label_test=to_categorical(label_test)
# Xcnn_test.shape()
len(Xcnn_test)

373

In [ ]:
print(len(Xcnn_test))
# len(Xcnn_test[0])
print(len(label_test))
print(len(label_test[0]))

373
373
4


In [ ]:
#cnn on text
from keras.models import Sequential
# from keras.optimizers import SGD
from keras import layers 
embedding_dim = 200
textcnnmodel = Sequential()
textcnnmodel.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
textcnnmodel.add(layers.Conv1D(128, 5, activation='relu'))
textcnnmodel.add(layers.GlobalMaxPooling1D())
textcnnmodel.add(layers.Dense(10, activation='relu'))
textcnnmodel.add(layers.Dense(4, activation='softmax'))
# opt = SGD(lr=0.01, momentum=0.9)
textcnnmodel.compile(optimizer='adam',
               loss='sparse_categorical_crossentropy',
               metrics=['accuracy'])
textcnnmodel.summary() 

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 50, 200)           712800    
                                                                 
 conv1d (Conv1D)             (None, 46, 128)           128128    
                                                                 
 global_max_pooling1d (Globa  (None, 128)              0         
 lMaxPooling1D)                                                  
                                                                 
 dense (Dense)               (None, 10)                1290      
                                                                 
 dense_1 (Dense)             (None, 4)                 44        
                                                                 
Total params: 842,262
Trainable params: 842,262
Non-trainable params: 0
__________________________________________________

In [ ]:
print(len(Xcnn_test))
print(len(label_test))
print(len(Xcnn_train))
print(len(label_train))


373
373
1116
1116


In [ ]:
print(np.shape(Xcnn_train))
print(np.shape(label_train))
print(np.shape(Xcnn_test))
print(np.shape(label_test))

(1116, 50)
(1116,)
(373, 50, 3564)
(373, 4)


In [ ]:
textcnnmodel.fit(Xcnn_train, label_train,
                     epochs=50,
                     verbose=True,
                     validation_data=(Xcnn_test, label_test),
                     batch_size=5)

# # loss, accuracy = textcnnmodel.evaluate(Xcnn_train, label_train, verbose=False)
# print("Training Accuracy: {:.4f}".format(accuracy))
# # loss, accuracy = textcnnmodel.evaluate(Xcnn_test, label_test, verbose=False)
# print("Testing Accuracy:  {:.4f}".format(accuracy)) 

Epoch 1/50
223/224 [============================>.] - ETA: 0s - loss: 1.2030 - accuracy: 0.4332WARNING:tensorflow:Model was constructed with shape (None, 50) for input KerasTensor(type_spec=TensorSpec(shape=(None, 50), dtype=tf.float32, name='embedding_input'), name='embedding_input', description="created by layer 'embedding_input'"), but it was called on an input with incompatible shape (None, 50, 3564).


ValueError: ignored